In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd://home/conchae/depolymerase/pdb_depolymerase_proteins_Bea \
/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea

rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase/Bea_sword2_out \
/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea  


 > Foldseek the domains

In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea"
path_pdb = f"{path_project}/Bea_sword2_out"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    with open(f"{path_project}/scan_domains.ppt.log","a+") as outfile : 
        dir_out = f"{path_project}/foldseek_Bea"
        protein_id = path_in.split("/")[-1].split(".pdb")[0]
        path_out = f"{dir_out}/{protein_id}.out"
        output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
        seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
        seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        press_out, press_err = seek_process.communicate()
        print (press_out, press_err)
        outfile.write(f"{protein_id}\n")

paths_domain = []
for file in os.listdir(f"{path_pdb}") :
    if file != "sword2.log" :
        n = 0
        for pred in os.listdir(f"{path_pdb}/{file}") :
            if "Protein_Units" in os.listdir(f"{path_pdb}/{file}/{pred}") : 
                n += 1
                for pdb_file in os.listdir(f"{path_pdb}/{file}/{pred}/Protein_Units") : 
                    if pdb_file[-3:] == "pdb" :
                        path = f"{path_pdb}/{file}/{pred}/Protein_Units/{pdb_file}"
                        paths_domain.append(path)
            else :
                continue
        if n == 0 :
            print(file)

if __name__ == '__main__':
    with ThreadPool(1) as p:
        p.map(seek_beta_helix, paths_domain)

> Foldseek the full 

In [ ]:
from multiprocessing.pool import ThreadPool
import subprocess
import os

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea"
path_pdb = f"{path_project}/Bea_sword2_out"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    with open(f"{path_project}/scan.ppt.log","a+") as outfile : 
        dir_out = f"{path_project}/foldseek_Bea_full"
        protein_id = path_in.split("/")[-1].split(".pdb")[0]
        path_out = f"{dir_out}/{protein_id}.out"
        output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
        seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
        seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        press_out, press_err = seek_process.communicate()
        print (press_out, press_err)
        outfile.write(f"{protein_id}\n")

paths_full_pdb = []
for file in os.listdir(f"{path_pdb}") :
    if file != "sword2.log" :
        n = 0
        for pred in os.listdir(f"{path_pdb}/{file}") :
            if "Protein_Units" in os.listdir(f"{path_pdb}/{file}/{pred}") : 
                n += 1
                path = f"{path_pdb}/{file}/{pred}/SWORD/{file}_A/{file}_A.pdb"
                paths_full_pdb.append(path)
            else :
                continue
        if n == 0 :
            print(file)

if __name__ == '__main__':
    with ThreadPool(1) as p:
        p.map(seek_beta_helix, paths_full_pdb)

***
### Scann the domains :

In [19]:
from tqdm import tqdm
import json
import sys
import os 
import pandas as pd 
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea"
dir_out_full = f"{path_project}/foldseek_Bea_full"
dir_out = f"{path_project}/foldseek_Bea"
path_swords_out = f"{path_project}/sword2_PhageDepo_pt2"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
dico_folds_77 = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

errors = {'A1l_00058_A_1_129_274.pdb',
 'AF-A0A089GJA0-F1-model_v4.pdb',
 'AF-A0A4V2M274-F1-model_v4.pdb',
 'AF-A0A7W9JCS6-F1-model_v4.pdb','AF-A0A658Y395-F1-model_v4.pdb'}

dico_out = {}
for file in os.listdir(dir_out) :
    protein = file.split("_A")[0]
    if protein not in dico_out : 
        tmp_list = []
        tmp_list.append(f"{dir_out}/{file}")
        dico_out[protein] = tmp_list
    else :
        dico_out[protein].append(f"{dir_out}/{file}")

length_prot = {}
for file in os.listdir(f"{path_pdb}") :
    if file != "sword2.log" :
        n = 0
        for pred in os.listdir(f"{path_pdb}/{file}") :
            if "Protein_Units" in os.listdir(f"{path_pdb}/{file}/{pred}") : 
                n += 1
                path = f"{path_pdb}/{file}/{pred}/SWORD/{file}_A/{file}_A.pdb"
                fasta_file = list(SeqIO.parse(f"{path_pdb}/{file}/{pred}/SWORD/{file}_A/{file}_A.fasta", "fasta"))[0]
                length_prot[file] = len(fasta_file)
            else :
                continue
        if n == 0 :
            print(file)

In [23]:
from tqdm import tqdm
import json
import sys
import os 
import pandas as pd 
from Bio import SeqIO

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro/Bea"
dir_out_full = f"{path_project}/foldseek_Bea_full"
dir_out = f"{path_project}/foldseek_Bea"
path_swords_out = f"{path_project}/sword2_PhageDepo_pt2"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
dico_folds_77 = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

errors = {'A1l_00058_A_1_129_274.pdb',
 'AF-A0A089GJA0-F1-model_v4.pdb',
 'AF-A0A4V2M274-F1-model_v4.pdb',
 'AF-A0A7W9JCS6-F1-model_v4.pdb','AF-A0A658Y395-F1-model_v4.pdb'}

dico_out = {}
for file in os.listdir(dir_out) :
    protein = file.split("_A")[0]
    if protein not in dico_out : 
        tmp_list = []
        tmp_list.append(f"{dir_out}/{file}")
        dico_out[protein] = tmp_list
    else :
        dico_out[protein].append(f"{dir_out}/{file}")

length_prot = {}
for file in os.listdir(f"{path_pdb}") :
    if file != "sword2.log" :
        n = 0
        for pred in os.listdir(f"{path_pdb}/{file}") :
            if "Protein_Units" in os.listdir(f"{path_pdb}/{file}/{pred}") : 
                n += 1
                path = f"{path_pdb}/{file}/{pred}/SWORD/{file}_A/{file}_A.pdb"
                fasta_file = list(SeqIO.parse(f"{path_pdb}/{file}/{pred}/SWORD/{file}_A/{file}_A.fasta", "fasta"))[0]
                length_prot[file] = len(fasta_file)
            else :
                continue
        if n == 0 :
            print(file)
            
dico_domain_result = {}
for protein in tqdm(dico_out) :
    the_winner = (0,0,0)
    len_protein = length_prot[protein]
    
    # Collecting data for each proteins
    results_full_prot = pd.read_csv(f"{dir_out_full}/{protein}_A.out" , sep = "\t" , names = header_seekfold).fillna("missing")
    results_full_prot = results_full_prot[~results_full_prot["target"].isin(errors)]
    for _,row in results_full_prot.iterrows():
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" :
            full_score = int(row["bits"])**1.2 / len_protein**0.40
            #full_score = int(row["bits"])
            target = row["target"]
            break
    the_full_prot = (full_score , f"len alignemnt : {row['alnlen']}" ,f"Full protein {protein}")
    for path in dico_out[protein] :
        if os.path.getsize(path) > 0 :
            results_df = pd.read_csv(f"{path}", sep = "\t" , names = header_seekfold).fillna("missing")
            results_df = results_df[results_df["target"] == target]
            if results_df.empty == False :
                aa_end = results_df["query"].values[0].split("_")[-1].split(".pdb")[0]
                aa_start = results_df["query"].values[0].split("_")[-2]
                PU_length = int(aa_end) - int(aa_start)
                score = int(results_df["bits"].values[0])**1.2 / PU_length**0.40
                #score = int(results_df["bits"].values[0]) 
                contender = (score , PU_length ,f"len alignemnt : {results_df['alnlen'].values[0]}",results_df["query"].values[0])
                if contender[0] > the_winner[0] :
                    the_winner = contender
                elif contender[0] == the_winner[0] :
                    if contender[1] < the_winner[1] :
                        the_winner = contender
                else :
                    pass
    #if the_full_prot[0] > (the_winner[0] + 25) :
    if the_full_prot[0] > (the_winner[0] + 2) :
        the_winner = the_full_prot 
    if the_winner[0] > 0 :
        dico_domain_result[protein] = the_winner

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:11<00:00,  4.53it/s]


In [25]:
import pprint
pp = pprint.PrettyPrinter(width = 250, sort_dicts = True, compact = True)
pp.pprint(dico_domain_result)


{'A1a_00014': (15.330334196433679, 357, 'len alignemnt : 340', 'A1a_00014_A_2_89_446.pdb'),
 'A1b_00048': (17.527661548047487, 328, 'len alignemnt : 390', 'A1b_00048_A_2_91_419.pdb'),
 'A1c_00034': (27.027613618990415, 490, 'len alignemnt : 555', 'A1c_00034_A_2_69_559.pdb'),
 'A1d_00009': (10.947704705280554, 467, 'len alignemnt : 530', 'A1d_00009_A_5_244_711.pdb'),
 'A1f_00012': (10.6870331422416, 496, 'len alignemnt : 438', 'A1f_00012_A_1_1_497.pdb'),
 'A1g_00057': (58.3968871485745, 498, 'len alignemnt : 579', 'A1g_00057_A_1_1_499.pdb'),
 'A1h_00013': (27.107253789165192, 408, 'len alignemnt : 653', 'A1h_00013_A_6_364_772.pdb'),
 'A1h_00021': (22.752300837262517, 75, 'len alignemnt : 79', 'A1h_00021_A_7_83_158.pdb'),
 'A1i_00041': (24.700627549408516, 404, 'len alignemnt : 441', 'A1i_00041_A_7_364_768.pdb'),
 'A1i_00049': (201.14458124708776, 472, 'len alignemnt : 458', 'A1i_00049_A_2_1_473.pdb'),
 'A1j_00040': (210.86784773864161, 355, 'len alignemnt : 352', 'A1j_00040_A_2_1_356.pd